# LSTM Autoencoder

2026-01-28

Reconstruction-based LSTM: encodes sequence, reconstructs it. High reconstruction error = anomaly.
Comparing full-size (100 units, SKAB config) vs tiny (16 units) for edge deployment.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import pandas as pd
import numpy as np
import time
import pickle
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, confusion_matrix
from tensorflow.keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_dir = Path('../data/raw/SKAB/data')

files = []
for folder in ['valve1', 'valve2', 'other']:
    files.extend(sorted((data_dir / folder).glob('*.csv')))

feature_cols = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current', 'Pressure',
                'Temperature', 'Thermocouple', 'Voltage', 'Volume Flow RateRMS']

len(files)

In [ ]:
TRAIN_SIZE = 400
N_STEPS = 10
EPOCHS = 100
BATCH_SIZE = 32
VAL_SPLIT = 0.1
Q = 0.99

def create_sequences(values, n_steps):
    output = []
    for i in range(len(values) - n_steps + 1):
        output.append(values[i:(i + n_steps)])
    return np.stack(output)

def build_lstm_ae(n_steps, n_features, units):
    inputs = Input(shape=(n_steps, n_features))
    encoded = LSTM(units, activation='relu')(inputs)
    decoded = RepeatVector(n_steps)(encoded)
    decoded = LSTM(units, activation='relu', return_sequences=True)(decoded)
    decoded = TimeDistributed(Dense(n_features))(decoded)
    model = Model(inputs, decoded)
    model.compile(optimizer='adam', loss='mae', metrics=['mse'])
    return model

def evaluate_lstm_ae(files, units, verbose=True):
    all_y_true, all_y_pred = [], []
    
    for i, f in enumerate(files):
        df = pd.read_csv(f, sep=';', parse_dates=['datetime'], index_col='datetime')
        X, y = df[feature_cols].values, df['anomaly'].values
        
        scaler = StandardScaler()
        X_train_sc = scaler.fit_transform(X[:TRAIN_SIZE])
        X_test_sc = scaler.transform(X[TRAIN_SIZE:])
        
        X_tr = create_sequences(X_train_sc, N_STEPS)
        X_te = create_sequences(X_test_sc, N_STEPS)
        
        tf.random.set_seed(0)
        np.random.seed(0)
        model = build_lstm_ae(N_STEPS, len(feature_cols), units)
        model.fit(X_tr, X_tr, validation_split=VAL_SPLIT, epochs=EPOCHS, batch_size=BATCH_SIZE,
                  verbose=0, shuffle=False, callbacks=[EarlyStopping(patience=5, verbose=0)])
        
        train_recon = model.predict(X_tr, verbose=0)
        residuals_train = np.sum(np.mean(np.abs(X_tr - train_recon), axis=1), axis=1)
        UCL = np.percentile(residuals_train, Q * 100) * 3 / 2
        
        test_recon = model.predict(X_te, verbose=0)
        residuals_test = np.sum(np.mean(np.abs(X_te - test_recon), axis=1), axis=1)
        
        # SKAB anomaly logic: flag if all samples in window are anomalous
        anomalous_data = residuals_test > UCL
        anomalous_indices = []
        for idx in range(N_STEPS - 1, len(X_te) - N_STEPS + 1):
            if np.all(anomalous_data[idx - N_STEPS + 1:idx]):
                anomalous_indices.append(idx)
        
        pred = np.zeros(len(y[TRAIN_SIZE:]))
        for idx in anomalous_indices:
            if idx < len(pred):
                pred[idx] = 1
        
        all_y_true.extend(y[TRAIN_SIZE:])
        all_y_pred.extend(pred)
        
        if verbose and (i+1) % 10 == 0:
            print(f'{i+1}/{len(files)} files')
    
    y_true, y_pred = np.array(all_y_true), np.array(all_y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    return {'f1': f1, 'far': fp/(fp+tn)*100, 'mar': fn/(fn+tp)*100,
            'size_kb': len(pickle.dumps(model))/1024, 'model': model}

In [ ]:
# full-size (100 units)
results_full = evaluate_lstm_ae(files, units=100)
print(f"F1: {results_full['f1']:.2f} (SKAB: 0.74), Size: {results_full['size_kb']:.0f} KB")

In [ ]:
# tiny (16 units)
results_tiny = evaluate_lstm_ae(files, units=16)
print(f"F1: {results_tiny['f1']:.2f} (full: {results_full['f1']:.2f}), Size: {results_tiny['size_kb']:.0f} KB")

In [ ]:
comparison = pd.DataFrame({
    'Model': ['Full (100 units)', 'Tiny (16 units)'],
    'F1': [results_full['f1'], results_tiny['f1']],
    'FAR %': [results_full['far'], results_tiny['far']],
    'MAR %': [results_full['mar'], results_tiny['mar']],
    'Size KB': [results_full['size_kb'], results_tiny['size_kb']]
})
comparison

In [ ]:
f1_drop = (results_full['f1'] - results_tiny['f1']) / results_full['f1'] * 100
size_reduction = (results_full['size_kb'] - results_tiny['size_kb']) / results_full['size_kb'] * 100
print(f"F1 drop: {f1_drop:.1f}%, Size reduction: {size_reduction:.1f}%")

In [ ]:
# latency
sample = pd.read_csv(files[5], sep=';', parse_dates=['datetime'], index_col='datetime')
sc = StandardScaler()
X_sc = sc.fit_transform(sample[feature_cols].values[:TRAIN_SIZE])
X_seq = create_sequences(X_sc, N_STEPS)
single = X_seq[:1]

tf.random.set_seed(0)
m_full = build_lstm_ae(N_STEPS, 8, 100)
m_full.fit(X_seq, X_seq, epochs=3, verbose=0)

tf.random.set_seed(0)
m_tiny = build_lstm_ae(N_STEPS, 8, 16)
m_tiny.fit(X_seq, X_seq, epochs=3, verbose=0)

m_full.predict(single, verbose=0)
m_tiny.predict(single, verbose=0)

times = []
for _ in range(500):
    t0 = time.perf_counter()
    m_full.predict(single, verbose=0)
    times.append(time.perf_counter() - t0)
lat_full = np.median(times) * 1000

times = []
for _ in range(500):
    t0 = time.perf_counter()
    m_tiny.predict(single, verbose=0)
    times.append(time.perf_counter() - t0)
lat_tiny = np.median(times) * 1000

print(f"Full: {lat_full:.1f} ms, Tiny: {lat_tiny:.1f} ms")

---

LSTM-AE more resilient to shrinking than Vanilla LSTM: ~94% size reduction with only ~3% F1 drop.
Best accuracy of all models tested (F1≈0.75), and tiny version still outperforms other methods.